<a href="https://colab.research.google.com/github/AnnaKudrina86659/book_of_recipes_bot/blob/main/telegram_recipes_bot_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🤖 Telegram-бот «Книга рецептов»

## 📋 О проекте

Этот бот в Telegram помогает быстро находить и получать рецепты блюд прямо в чате.  
Пользователь может:
- Получить **случайный рецепт дня**.
- Запросить **вегетарианский рецепт**.
- Искать рецепты по **названию** блюда или по **ингредиентам**.
- Просматривать рецепт **пошагово** и переключаться между шагами.

**Источник рецептов:** база была **создана вручную** и **дополнена парсингом** с сайта «https://www.edimdoma.ru/retsepty».  
Хранилище рецептов — `SQLite3`. Синхронизация состояния (и при необходимости баз) — через **Yandex Object Storage (S3-совместимый)**.

> ℹ️ Проект предназначен для демонстрации работы Telegram-бота, работы со state сохранением, пагинацией в поиске и интеграцией с S3-хранилищем.



## ⚙️ Функционал

- **Главное меню** с тремя действиями: `Рецепт дня`, `Вегетарианский рецепт`, `Поиск рецепта`.
- **Случайный рецепт** — выдача одного случайного рецепта из базы.
- **Вегетарианский фильтр** — выбор случайного рецепта, где `is_vegetarian = 1`.
- **Поиск с пагинацией** — поиск по названию/ингредиентам + разбиение результатов по 5 на страницу.
- **Пошаговое приготовление** — кнопки `Шаг N`, `◀ Назад`, `Вперед ▶`, `К рецепту`.
- **Сохранение состояния** пользователя в `user_states.db` + восстановление при новом `/start`.
- **Фоновая очистка** сессий старше 7 дней.
- **Синхронизация с S3** (скачивание/загрузка `recipes.db` и `user_states.db`).



## 🗺 Архитектура и данные

- **SQLite**
  - `recipes.db` — база рецептов (ожидаемые поля: `title`, `time_for_cook`, `ingredients`, `recipe_url`, `links_images`, `steps`, `is_vegetarian`).
  - `user_states.db` — состояния пользователей (`last_activity`, `current_state`, сохраненные результаты поиска, текущий шаг рецепта и пр.).
- **S3 (Yandex Object Storage)** — хранение/синхронизация `*.db` файлов.
- **pytelegrambotapi (`telebot`)** — связка с Telegram Bot API.
- **Поток очистки** — периодически удаляет устаревшие записи в `user_states`.

> Примечание: структура `recipes` должна соответствовать SQL-запросам в коде (см. разделы «Рецепт дня», «Вегетарианский рецепт», «Поиск»).



## 📦 Установка зависимостей
Выполните один раз:


In [ ]:

!pip install pytelegrambotapi boto3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 6.3 MB/s eta 0:00:00



## 🔐 Безопасность и переменные окружения

**Токен бота и ключи облака не должны храниться в репозитории.**  
В этом ноутбуке токен считывается из переменной окружения `TELEGRAM_BOT_TOKEN`.  
Ключи S3 — из `YC_ACCESS_KEY_ID` и `YC_SECRET_ACCESS_KEY`.

В Colab можно временно задать значения так (они не попадут в репозиторий, если не сохранять ноутбук с введенными значениями):


In [ ]:

#@title 🔑 Введите секреты окружения (по желанию)
import os

# Либо задайте через интерфейс Colab -> Edit -> Notebook settings -> Variables
# Ниже — безопасная временная установка на время сессии:
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN") or ""  # вставьте токен сюда строкой либо задайте в переменных среды
YC_ACCESS_KEY_ID = os.getenv("YC_ACCESS_KEY_ID") or ""
YC_SECRET_ACCESS_KEY = os.getenv("YC_SECRET_ACCESS_KEY") or ""

# При необходимости раскомментируйте и введите руками (не сохраняйте ноутбук с заполненными секретами):
# TELEGRAM_BOT_TOKEN = input("Введите TELEGRAM_BOT_TOKEN: ").strip() or TELEGRAM_BOT_TOKEN
# YC_ACCESS_KEY_ID = input("Введите YC_ACCESS_KEY_ID: ").strip() or YC_ACCESS_KEY_ID
# YC_SECRET_ACCESS_KEY = input("Введите YC_SECRET_ACCESS_KEY: ").strip() or YC_SECRET_ACCESS_KEY

os.environ["TELEGRAM_BOT_TOKEN"] = TELEGRAM_BOT_TOKEN
os.environ["YC_ACCESS_KEY_ID"] = YC_ACCESS_KEY_ID
os.environ["YC_SECRET_ACCESS_KEY"] = YC_SECRET_ACCESS_KEY

print("Ок: переменные окружения выставлены (значения не отображаются).")



## 📚 Импорты


In [ ]:

import telebot
from telebot import types
import sqlite3
import re
import os
import time
import threading
from datetime import datetime, timedelta
import ast
import boto3



## 🔧 Настройки и инициализация бота и S3

- Токен берется из `os.getenv("TELEGRAM_BOT_TOKEN")` (если не задан — будет ошибка).
- S3-клиент под Yandex Object Storage.
- Имена бакета и путей к базам.


In [ ]:

# ======================================
#            НАСТРОЙКИ
# ======================================
TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")  # токен только из переменной окружения
if not TOKEN:
    raise ValueError("❌ TELEGRAM_BOT_TOKEN не установлен. Задайте его перед запуском.")

bot = telebot.TeleBot(token=TOKEN, parse_mode=None)

# Конфигурация S3-клиента (Yandex Object Storage, S3-совместимый)
session = boto3.session.Session(
    aws_access_key_id=os.getenv("YC_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("YC_SECRET_ACCESS_KEY"),
    region_name="ru-central1"
)
s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net'
)

BUCKET_NAME = "bot-bucket-book-of-recipes"
RECIPES_DB_OBJECT = "recipes.db"
STATE_DB_OBJECT = "user_states.db"

RECIPES_DB_PATH = "/tmp/recipes.db"
STATE_DB_PATH = "/tmp/user_states.db"



## ☁️ Утилиты работы с Object Storage


In [ ]:

# ======================================
#       УТИЛИТЫ РАБОТЫ С S3
# ======================================
def download_file_if_needed(bucket_name, object_name, local_path):
    """Скачиваем файл из Object Storage, если локального ещё нет."""
    if os.path.exists(local_path):
        return
    try:
        # Пробуем скачать
        s3.download_file(bucket_name, object_name, local_path)
        print(f"✅ {object_name} скачан из бакета {bucket_name} в {local_path}")
    except Exception as e:
        # Если файла действительно нет в бакете — это не критично для state DB (мы создадим пустой)
        print(f"⚠ Не удалось скачать {object_name} из бакета {bucket_name}: {e}")

def upload_file(local_path, bucket_name, object_name):
    """Загружаем локальный файл в Object Storage."""
    if not os.path.exists(local_path):
        print(f"⚠ Локальный файл {local_path} отсутствует — нечего загружать")
        return
    try:
        s3.upload_file(local_path, bucket_name, object_name)
        print(f"✅ {object_name} обновлён в бакете {bucket_name}")
    except Exception as e:
        print(f"❌ Ошибка загрузки {object_name} в бакет {bucket_name}: {e}")



## 🧠 Глобальные структуры памяти


In [ ]:

# ======================================
#      ГЛОБАЛЬНЫЕ СТРУКТУРЫ ПАМЯТИ
# ======================================
user_steps = {}         # {chat_id: {'steps': [...], 'current_step': int, 'recipe_data': tuple, 'vegetarian': bool}}
user_search_state = {}  # {chat_id: "waiting_for_search_query"}
user_search_results = {}# {chat_id: {'results': [...], 'page': int, 'query': str}}
user_keyboards = {}     # для восстановления клавиатур
LAST_ACTIVITY = {}      # {chat_id: unix_timestamp}



## 🗄 Инициализация и подключение к базам (`recipes.db` и `user_states.db`)

- `get_recipes_connection()` — скачивает `recipes.db` из S3 (если нет локально) и открывает соединение.
- `get_state_connection()` — создаёт/открывает `user_states.db`, создает таблицу `user_states` (если нет), включает WAL, и возвращает соединение.


In [ ]:

# ======================================
#   ИНИЦИАЛИЗАЦИЯ/ПОДКЛЮЧЕНИЕ К БАЗАМ
# ======================================
def get_recipes_connection():
    # Рецепты должны быть заранее загружены в бакет под именем recipes.db
    download_file_if_needed(BUCKET_NAME, RECIPES_DB_OBJECT, RECIPES_DB_PATH)
    try:
        return sqlite3.connect(RECIPES_DB_PATH)
    except Exception as e:
        print(f"❌ Ошибка подключения к recipes.db: {e}")
        return None

def get_state_connection():
    # Пытаемся скачать state.db из бакета (не обязательно существует)
    download_file_if_needed(BUCKET_NAME, STATE_DB_OBJECT, STATE_DB_PATH)

    # Создаём директорию и файл, если их нет
    try:
        os.makedirs(os.path.dirname(STATE_DB_PATH), exist_ok=True)
    except Exception:
        pass

    if not os.path.exists(STATE_DB_PATH):
        open(STATE_DB_PATH, 'w').close()
        try:
            os.chmod(STATE_DB_PATH, 0o666)
        except Exception:
            pass

    try:
        conn = sqlite3.connect(STATE_DB_PATH)
        conn.execute("PRAGMA journal_mode=WAL")   # Улучшаем параллельный доступ
        conn.execute("PRAGMA synchronous=NORMAL")
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS user_states (
                chat_id INTEGER PRIMARY KEY,
                last_activity TEXT,
                current_state TEXT,
                recipe_title TEXT,
                time_for_cook TEXT,
                ingredients TEXT,
                recipe_url TEXT,
                links_images TEXT,
                steps TEXT,
                current_step INTEGER,
                search_query TEXT,
                search_results TEXT,
                search_page INTEGER,
                is_vegetarian INTEGER
            )
        ''')
        conn.commit()
        return conn
    except Exception as e:
        print(f"❌ Ошибка создания/подключения user_states.db: {e}")
        return None



## ✏️ Форматирование данных перед отправкой

- `format_time()` — заменяет «час/минуты» на компактный `1h⏱/15min⏱`.
- `format_ingredients()` — превращает список ингредиентов в маркированный список.
- `format_steps()` — извлекает шаги вида `1. ... 2. ...` в список.


In [ ]:

# ======================================
#         ФОРМАТИРОВАНИЕ ВЫВОДА
# ======================================
def format_time(time_str):
    if not time_str:
        return ""
    s = time_str
    s = re.sub(r'(\d+)\s*(часов|часа|час)\s*(и\s*)?(\d+)\s*(минут[а-я]*)', r'\1h⏱ \4min⏱', s, flags=re.IGNORECASE)
    s = re.sub(r'(\d+)\s*(час[а-я]*)', r'\1h⏱', s, flags=re.IGNORECASE)
    s = re.sub(r'(\d+)\s*(минут[а-я]*)', r'\1min⏱', s, flags=re.IGNORECASE)
    s = re.sub(r'\b(час[а-я]*)\b', '1h⏱', s, flags=re.IGNORECASE)
    s = re.sub(r'\b(минут[а-я]*)\b', '1min⏱', s, flags=re.IGNORECASE)
    return s

def format_ingredients(ingredients):
    if not ingredients:
        return ""
    return "\n".join(f"• {it.strip()}" for it in ingredients.split(',') if it.strip())

def format_steps(steps_text):
    if not steps_text:
        return []
    matches = re.findall(r'\s*(\d+)\.\s*(.*?)(?=\s*\d+\.|$)', steps_text, re.DOTALL)
    return [m[1].strip().rstrip(',') for m in matches]



## 💾 Сохранение и загрузка состояния

- `save_user_state(chat_id)` — складывает текущее состояние в `user_states.db` и синхронизирует с S3.
- `load_user_state(chat_id)` — загружает состояние пользователя (включая результаты поиска).
- `update_activity(chat_id)` — обновляет метку активности и триггерит сохранение.


In [ ]:

# ======================================
#   СОХРАНЕНИЕ/ЗАГРУЗКА СОСТОЯНИЙ В БД
# ======================================
def save_user_state(chat_id):
    """Сохраняет текущее состояние пользователя и загружает файл в бакет."""
    conn = get_state_connection()
    if not conn:
        print("⚠ Не удалось получить соединение с user_states.db в save_user_state")
        return
    try:
        cursor = conn.cursor()
        # Подготавливаем поля
        now_iso = datetime.now().isoformat()
        current_state = None
        recipe_title = time_for_cook = ingredients = recipe_url = links_images = steps = None
        current_step = search_query = search_results = search_page = None
        is_vegetarian = 0

        if chat_id in user_steps:
            rd = user_steps[chat_id]['recipe_data']
            current_state = 'recipe'
            recipe_title, time_for_cook, ingredients, recipe_url, links_images, steps = rd
            current_step = user_steps[chat_id].get('current_step', 0)
            is_vegetarian = 1 if user_steps[chat_id].get('vegetarian') else 0
        elif chat_id in user_search_results:
            current_state = 'search'
            search_query = user_search_results[chat_id].get('query')
            try:
                search_results = str(user_search_results[chat_id].get('results'))
            except Exception:
                search_results = None
            search_page = user_search_results[chat_id].get('page', 0)

        cursor.execute('''
            INSERT INTO user_states (
                chat_id, last_activity, current_state,
                recipe_title, time_for_cook, ingredients,
                recipe_url, links_images, steps,
                current_step, search_query, search_results,
                search_page, is_vegetarian
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ON CONFLICT(chat_id) DO UPDATE SET
                last_activity=excluded.last_activity,
                current_state=excluded.current_state,
                recipe_title=excluded.recipe_title,
                time_for_cook=excluded.time_for_cook,
                ingredients=excluded.ingredients,
                recipe_url=excluded.recipe_url,
                links_images=excluded.links_images,
                steps=excluded.steps,
                current_step=excluded.current_step,
                search_query=excluded.search_query,
                search_results=excluded.search_results,
                search_page=excluded.search_page,
                is_vegetarian=excluded.is_vegetarian
        ''', (
            chat_id, now_iso, current_state,
            recipe_title, time_for_cook, ingredients,
            recipe_url, links_images, steps,
            current_step, search_query, search_results,
            search_page, is_vegetarian
        ))
        conn.commit()
    except Exception as e:
        print(f"❌ Ошибка save_user_state для {chat_id}: {e}")
    finally:
        try:
            conn.close()
        except Exception:
            pass

    # Синхронизируем файл в Object Storage
    try:
        upload_file(STATE_DB_PATH, BUCKET_NAME, STATE_DB_OBJECT)
    except Exception as e:
        print(f"⚠ Ошибка upload_file в save_user_state: {e}")

def load_user_state(chat_id):
    """Загружает состояние одного пользователя (без восстановления оперативной памяти)"""
    conn = get_state_connection()
    if not conn:
        return None
    try:
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM user_states WHERE chat_id = ?', (chat_id,))
        row = cursor.fetchone()
        if not row:
            return None
        cols = ['chat_id','last_activity','current_state','recipe_title','time_for_cook','ingredients',
                'recipe_url','links_images','steps','current_step','search_query','search_results','search_page','is_vegetarian']
        state = dict(zip(cols, row))
        if state.get('search_results'):
            try:
                state['search_results'] = ast.literal_eval(state['search_results'])
            except Exception:
                state['search_results'] = None
        return state
    except Exception as e:
        print(f"❌ Ошибка load_user_state для {chat_id}: {e}")
        return None
    finally:
        try:
            conn.close()
        except Exception:
            pass

# ======================================
#   Фиксированный update_activity
# ======================================
def update_activity(chat_id):
    """Обновляет время активности в памяти и сохраняет состояние в БД."""
    LAST_ACTIVITY[chat_id] = time.time()
    try:
        save_user_state(chat_id)
    except Exception as e:
        print(f"⚠ Ошибка при update_activity save_user_state: {e}")



## 🖥 Интерфейс бота и обработчики меню

- Главное меню с кнопками.
- Восстановление состояния при `/start` (если активность была недавно).
- Подача рецептов и переключение между рецептами.


In [ ]:

# ======================================
#     ИНТЕРФЕЙС БОТА: меню и т.д.
# ======================================
def send_keyboard(chat_id, text="Привет, чем я могу тебе помочь?"):
    kb = types.ReplyKeyboardMarkup(row_width=1, resize_keyboard=True)
    kb.add(types.KeyboardButton('Рецепт дня'),
           types.KeyboardButton('Вегетарианский рецепт'),
           types.KeyboardButton('Поиск рецепта'))
    user_keyboards[chat_id] = {'type':'main_menu'}
    bot.send_message(chat_id, text=text, reply_markup=kb)

@bot.message_handler(commands=['start'])
def handle_start(message):
    update_activity(message.chat.id)
    # Попробуем восстановить состояние из БД (без падения)
    state = load_user_state(message.chat.id)
    if state:
        try:
            # проверка свежести
            if state.get('last_activity'):
                last = datetime.fromisoformat(state['last_activity'])
                if (datetime.now() - last) <= timedelta(days=1):
                    if state.get('current_state') == 'recipe' and state.get('recipe_title'):
                        recipe_data = (
                            state.get('recipe_title'),
                            state.get('time_for_cook'),
                            state.get('ingredients'),
                            state.get('recipe_url'),
                            state.get('links_images'),
                            state.get('steps')
                        )
                        user_steps[message.chat.id] = {
                            'recipe_data': recipe_data,
                            'current_step': state.get('current_step', 0),
                            'steps': format_steps(state.get('steps') or ""),
                            'vegetarian': bool(state.get('is_vegetarian'))
                        }
                        return send_recipe(message.chat.id, recipe_data, state.get('is_vegetarian'))
                    elif state.get('current_state') == 'search' and state.get('search_results'):
                        user_search_results[message.chat.id] = {
                            'results': state.get('search_results') or [],
                            'page': state.get('search_page', 0),
                            'query': state.get('search_query')
                        }
                        return show_search_results_page(message.chat.id, state.get('search_page', 0))
        except Exception as e:
            print(f"⚠ Ошибка восстановления при /start: {e}")

    send_keyboard(message.chat.id, "Привет! Я бот с рецептами. Чем могу помочь?")

@bot.message_handler(func=lambda m: m.text in ['Рецепт дня', 'Вегетарианский рецепт', 'Поиск рецепта'])
def handle_main_menu_buttons(message):
    update_activity(message.chat.id)
    if message.text == "Рецепт дня":
        send_random_recipe(message)
    elif message.text == 'Вегетарианский рецепт':
        send_vegetarian_recipe(message)
    elif message.text == 'Поиск рецепта':
        handle_search_button(message)

def send_recipe(chat_id, recipe, is_vegetarian=False):
    title, time_for_cook, ingredients, url, links_images, steps = recipe
    title = (title or "").capitalize()
    user_steps[chat_id] = {
        'steps': format_steps(steps or ""),
        'current_step': 0,
        'recipe_data': recipe
    }
    if is_vegetarian:
        user_steps[chat_id]['vegetarian'] = True

    msg = (f"<b>{title}</b>\n\n"
           f"⏱ <b>Время готовки:</b> {format_time(time_for_cook or '')}\n\n"
           f"🍽 <b>Ингредиенты:</b>\n{format_ingredients(ingredients or '')}\n\n"
           f"🔗 <a href='{url}'>Полный рецепт</a>")

    step_buttons = [types.KeyboardButton(f"Шаг {i+1}") for i in range(len(user_steps[chat_id]['steps']))]
    other_button = types.KeyboardButton('Другой вегетарианский рецепт' if is_vegetarian else 'Другой рецепт')

    kb = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=4)
    if step_buttons:
        kb.add(*step_buttons)
    kb.add(other_button, types.KeyboardButton('Главное меню'))

    user_keyboards[chat_id] = {'type':'recipe', 'data': recipe}

    # сохраняем состояние
    update_activity(chat_id)

    # отправка
    if links_images:
        try:
            bot.send_photo(chat_id, photo=links_images, caption=msg, parse_mode='HTML', reply_markup=kb)
        except Exception as e:
            print(f"⚠ Ошибка отправки фото: {e}")
            bot.send_message(chat_id, msg, parse_mode='HTML', reply_markup=kb)
    else:
        bot.send_message(chat_id, msg, parse_mode='HTML', reply_markup=kb)

def send_random_recipe(msg):
    conn = get_recipes_connection()
    if not conn:
        return bot.send_message(msg.chat.id, "⚠ Ошибка подключения к базе рецептов")
    try:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT title, time_for_cook, ingredients, recipe_url, links_images, steps
            FROM recipes
            ORDER BY RANDOM()
            LIMIT 1
        """)
        recipe = cursor.fetchone()
        if recipe:
            send_recipe(msg.chat.id, recipe)
        else:
            bot.send_message(msg.chat.id, "⚠ Рецепты не найдены в базе")
    except Exception as e:
        print(f"❌ Ошибка send_random_recipe: {e}")
        bot.send_message(msg.chat.id, "⚠ Ошибка при получении рецепта")
    finally:
        try:
            conn.close()
        except Exception:
            pass

def send_vegetarian_recipe(msg):
    conn = get_recipes_connection()
    if not conn:
        return bot.send_message(msg.chat.id, "⚠ Ошибка подключения к базе рецептов")
    try:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT title, time_for_cook, ingredients, recipe_url, links_images, steps
            FROM recipes
            WHERE is_vegetarian = 1
            ORDER BY RANDOM()
            LIMIT 1
        """)
        recipe = cursor.fetchone()
        if recipe:
            send_recipe(msg.chat.id, recipe, is_vegetarian=True)
        else:
            bot.send_message(msg.chat.id, "⚠ Вегетарианские рецепты не найдены")
    except Exception as e:
        print(f"❌ Ошибка send_vegetarian_recipe: {e}")
        bot.send_message(msg.chat.id, "⚠ Ошибка при получении рецепта")
    finally:
        try:
            conn.close()
        except Exception:
            pass

@bot.message_handler(func=lambda m: m.text in ['Другой рецепт', 'Другой вегетарианский рецепт', 'К рецепту', 'Главное меню'])
def handle_recipe_buttons(message):
    update_activity(message.chat.id)
    if message.text == 'Другой вегетарианский рецепт':
        send_vegetarian_recipe(message)
    elif message.text == 'Другой рецепт':
        send_random_recipe(message)
    elif message.text == 'К рецепту':
        back_to_recipe(message)
    elif message.text == 'Главное меню':
        handle_main_menu(message)

def handle_main_menu(message):
    update_activity(message.chat.id)
    try:
        bot.clear_step_handler_by_chat_id(message.chat.id)
    except Exception:
        pass
    user_search_state.pop(message.chat.id, None)
    user_search_results.pop(message.chat.id, None)
    user_steps.pop(message.chat.id, None)
    # удаляем запись из БД (опционально)
    try:
        conn = get_state_connection()
        if conn:
            cur = conn.cursor()
            cur.execute('DELETE FROM user_states WHERE chat_id = ?', (message.chat.id,))
            conn.commit()
            conn.close()
            upload_file(STATE_DB_PATH, BUCKET_NAME, STATE_DB_OBJECT)
    except Exception as e:
        print(f"⚠ Ошибка удаления состояния при handle_main_menu: {e}")
    send_keyboard(message.chat.id, "Вы вернулись в главное меню")



## 🔎 Поиск рецептов — особенности реализации

- Включен набор **стоп-слов** (`STOP_WORDS`), чтобы не учитывать служебные слова.
- Поиск выполняется одновременно по `title` **и** по `ingredients` через `LIKE`.
- Логика условий — `OR` между терминами (широкий охват).
- Пагинация — **по 5 результатов на страницу**; навигация кнопками `⬅ Назад` и `Далее ➡`.
- Режим `ForceReply` для ввода текста запроса прямо в диалоге.


In [ ]:

# ===== Поиск рецептов =====
@bot.message_handler(func=lambda m: m.text == "Поиск рецепта")
def handle_search_button(message):
    update_activity(message.chat.id)
    msg = bot.send_message(message.chat.id, "🔍 Введите название блюда или ингредиент:", reply_markup=types.ForceReply(selective=True))
    user_search_state[message.chat.id] = "waiting_for_search_query"
    bot.register_next_step_handler(msg, process_search_query)

def process_search_query(message):
    chat_id = message.chat.id
    update_activity(chat_id)
    query = (message.text or "").strip()

    STOP_WORDS = {'с','и','или','из','для','в','на','по','со','без','от','до','как','но','за','к','а','рецепт','блюдо','приготовить','сделать','чего','чем'}

    if not query or query in ['Рецепт дня', 'Вегетарианский рецепт', 'Поиск рецепта']:
        return send_keyboard(chat_id, "Пожалуйста, введите поисковый запрос")

    user_search_state.pop(chat_id, None)

    terms = [t.strip(" ,.!?").lower() for t in query.split() if t.strip(" ,.!?").lower() not in STOP_WORDS]
    if not terms:
        return send_keyboard(chat_id, "⚠ Введите более конкретный запрос")

    conn = get_recipes_connection()
    if not conn:
        return send_keyboard(chat_id, "⚠ Ошибка подключения к базе рецептов")
    try:
        cursor = conn.cursor()
        title_cond = " OR ".join(["LOWER(title) LIKE ?" for _ in terms])
        ingr_cond = " OR ".join(["LOWER(ingredients) LIKE ?" for _ in terms])
        params = [f"%{t}%" for t in terms] * 2
        cursor.execute(f"""
            SELECT title, time_for_cook, ingredients, recipe_url, links_images, steps
            FROM recipes
            WHERE ({title_cond}) OR ({ingr_cond})
        """, params)
        results = cursor.fetchall()
        if results:
            user_search_results[chat_id] = {'results': results, 'page': 0, 'query': query}
            show_search_results_page(chat_id, 0)
        else:
            send_keyboard(chat_id, f"😕 По запросу '{query}' ничего не найдено")
    except Exception as e:
        print(f"❌ Ошибка process_search_query: {e}")
        send_keyboard(chat_id, "⚠ Произошла ошибка при поиске")
    finally:
        try:
            conn.close()
        except Exception:
            pass

def show_search_results_page(chat_id, page):
    if not (sd := user_search_results.get(chat_id)):
        return
    results = sd['results']
    page_results = results[page*5:(page+1)*5]
    total_pages = (len(results)+4)//5
    kb = types.ReplyKeyboardMarkup(row_width=1, resize_keyboard=True)
    for i, r in enumerate(page_results):
        kb.add(types.KeyboardButton(f"{i+1}. {r[0].capitalize()}"))
    nav = []
    if page > 0:
        nav.append(types.KeyboardButton('⬅ Назад'))
    if (page+1)*5 < len(results):
        nav.append(types.KeyboardButton('Далее ➡'))
    if nav:
        kb.add(*nav)
    kb.add(types.KeyboardButton('Главное меню'))
    user_search_results[chat_id]['page'] = page
    user_keyboards[chat_id] = {'type':'search_results', 'data': {'page': page, 'query': sd.get('query')}}
    update_activity(chat_id)
    bot.send_message(
        chat_id,
        f"🔍 Результаты поиска '{sd.get('query')}' (стр. {page+1}/{total_pages}):\n" +
        "\n".join(f"{i+1}. {r[0].capitalize()}" for i,r in enumerate(page_results)),
        reply_markup=kb
    )

@bot.message_handler(func=lambda m: m.text in ['⬅ Назад', 'Далее ➡'] and m.chat.id in user_search_results)
def handle_page_navigation(message):
    update_activity(message.chat.id)
    sd = user_search_results.get(message.chat.id)
    if not sd:
        return
    page = sd['page']
    newp = page - 1 if message.text == '⬅ Назад' else page + 1
    show_search_results_page(message.chat.id, newp)

@bot.message_handler(func=lambda m: m.text.split(". ")[0].isdigit() and m.chat.id in user_search_results)
def handle_recipe_selection(message):
    update_activity(message.chat.id)
    try:
        chat_id = message.chat.id
        choice = int(message.text.split(". ")[0]) - 1
        sd = user_search_results[chat_id]
        absolute = sd['page']*5 + choice
        if 0 <= absolute < len(sd['results']):
            send_recipe(chat_id, sd['results'][absolute])
    except Exception as e:
        print(f"❌ Ошибка handle_recipe_selection: {e}")
        send_keyboard(message.chat.id, "❌ Ошибка. Попробуйте снова")



## 🥣 Пошаговое приготовление — логика

- Кнопки `Шаг N` создаются динамически по количеству шагов в рецепте.
- Навигация внутри шага: `◀ Назад` / `Вперед ▶`.
- Кнопка `К рецепту` возвращает к карточке рецепта с ингредиентами и ссылкой.


In [ ]:

# ===== Шаги приготовления =====
@bot.message_handler(func=lambda m: m.text and m.text.startswith('Шаг '))
def handle_step(message):
    update_activity(message.chat.id)
    if message.chat.id not in user_steps:
        return
    try:
        step_num = int(message.text.split()[1]) - 1
        steps = user_steps[message.chat.id]['steps']
        if 0 <= step_num < len(steps):
            user_steps[message.chat.id]['current_step'] = step_num
            show_step(message.chat.id, step_num, steps)
    except Exception:
        pass

def show_step(chat_id, step_num, steps):
    kb = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=3)
    if step_num > 0:
        kb.add(types.KeyboardButton("◀ Назад"))
    if step_num < len(steps)-1:
        kb.add(types.KeyboardButton("Вперед ▶"))
    kb.add(types.KeyboardButton('К рецепту'))
    user_keyboards[chat_id] = {'type':'steps','data': {'current_step': step_num, 'steps': steps}}
    update_activity(chat_id)
    bot.send_message(chat_id, f"<b>Шаг {step_num+1}:</b>\n{steps[step_num]}", parse_mode='HTML', reply_markup=kb)

@bot.message_handler(func=lambda m: m.text in ["◀ Назад", "Вперед ▶"])
def handle_step_navigation(message):
    update_activity(message.chat.id)
    if message.chat.id not in user_steps:
        return
    cur = user_steps[message.chat.id]['current_step']
    steps = user_steps[message.chat.id]['steps']
    if message.text == "◀ Назад" and cur > 0:
        new = cur - 1
    elif message.text == "Вперед ▶" and cur < len(steps)-1:
        new = cur + 1
    else:
        return
    user_steps[message.chat.id]['current_step'] = new
    show_step(message.chat.id, new, steps)

def back_to_recipe(message):
    update_activity(message.chat.id)
    if message.chat.id not in user_steps:
        return
    rd = user_steps[message.chat.id]['recipe_data']
    send_recipe(message.chat.id, rd)



## 🧯 Fallback-обработчик

Если текст не распознан, бот пытается восстановить релевантную клавиатуру (главное меню / рецепт / результаты поиска / шаги).


In [ ]:

# ===== Fallback =====
@bot.message_handler(func=lambda m: True)
def handle_unrecognized(message):
    update_activity(message.chat.id)
    if message.chat.id in user_keyboards:
        kb = user_keyboards[message.chat.id]
        try:
            if kb['type'] == 'main_menu':
                send_keyboard(message.chat.id)
                return
            if kb['type'] == 'recipe':
                send_recipe(message.chat.id, kb['data'])
                return
            if kb['type'] == 'search_results':
                show_search_results_page(message.chat.id, kb['data']['page'])
                return
            if kb['type'] == 'steps':
                show_step(message.chat.id, kb['data']['current_step'], kb['data']['steps'])
                return
        except Exception as e:
            print(f"⚠ Ошибка восстановления клавиатуры в fallback: {e}")
    send_keyboard(message.chat.id, "Я не понял ваш запрос. Выберите действие из меню:")



## 🧹 Очистка старых сессий

Фоновая задача удаляет записи старше 7 дней из `user_states`.  
При ошибках прав пытается скорректировать права на файл БД.


In [ ]:

# ======================================
#    Фоновая чистка старых записей
# ======================================
def cleanup_old_sessions():
    """Очистка старых сессий с обработкой ошибок прав"""
    while True:
        try:
            conn = get_state_connection()
            if conn:
                try:
                    week_ago = (datetime.now() - timedelta(days=7)).isoformat()
                    conn.execute("DELETE FROM user_states WHERE last_activity < ?", (week_ago,))
                    conn.commit()
                finally:
                    conn.close()

                # Пытаемся сохранить изменения в Object Storage
                try:
                    upload_file(STATE_DB_PATH, BUCKET_NAME, STATE_DB_OBJECT)
                except Exception as upload_error:
                    print(f"⚠ Upload failed: {upload_error}")
        except sqlite3.OperationalError as e:
            if "readonly" in str(e).lower():
                print("⚠ DB is readonly, trying to fix permissions...")
                try:
                    os.chmod(STATE_DB_PATH, 0o666)
                except Exception as chmod_error:
                    print(f"❌ Failed to fix permissions: {chmod_error}")
            else:
                print(f"❌ DB Error: {e}")
        except Exception as e:
            print(f"⚠ Cleanup error: {e}")

        time.sleep(86400)  # 24 часа



## ▶️ Запуск бота

- При первом старте файл `user_states.db` создается при необходимости.
- Запускается поток очистки.
- Запускается `bot.infinity_polling()`.

> 💡 В Colab выполнение `infinity_polling()` блокирует ячейку — это ожидаемо для long-running процессов.


In [ ]:

# ======================================
#          Запуск бота
# ======================================
if __name__ == '__main__':
    # Проверка и создание БД при старте
    if not os.path.exists(STATE_DB_PATH):
        with open(STATE_DB_PATH, 'w') as f:
            f.close()
        try:
            os.chmod(STATE_DB_PATH, 0o666)
        except Exception:
            pass

    # Запускаем очистку в отдельном потоке
    cleaner = threading.Thread(target=cleanup_old_sessions, daemon=True)
    cleaner.start()

    # Основной цикл бота
    bot.infinity_polling()



## 🧩 Примечания и рекомендации

### Структура таблицы `recipes` (ожидаемая)
Минимально требуемые поля для текущих SQL-запросов:
- `title` — название блюда (TEXT)
- `time_for_cook` — строка формата времени («1 час 20 минут» и т.п.)
- `ingredients` — строка со списком ингредиентов, разделенных запятыми
- `recipe_url` — ссылка на полный рецепт
- `links_images` — ссылка на изображение (опционально)
- `steps` — текст шагов в формате `1. ... 2. ... 3. ...`
- `is_vegetarian` — `INTEGER` (0/1)

### Деплой
- Создайте Telegram-бота у @BotFather и получите токен.
- Загрузите `recipes.db` в бакет `bot-bucket-book-of-recipes` под ключом `recipes.db`.
- На сервере/в контейнере перед запуском экспортируйте переменные окружения:
  - `TELEGRAM_BOT_TOKEN`
  - `YC_ACCESS_KEY_ID`
  - `YC_SECRET_ACCESS_KEY`
- Запустите скрипт (или ноутбук).

## ☁️ Развертывание

Проект тестировался и запускался на виртуальной машине в [Yandex Cloud](https://cloud.yandex.ru/).  
Для работы был создан Linux-сервер (Ubuntu), на котором:
- Установлен Python и необходимые зависимости.
- Настроена среда с переменными окружения (токены и ключи).
- Запущен скрипт бота в фоновом режиме через screen`/`tmux или systemd.
- Настроено автоматическое обновление базы рецептов через синхронизацию с Yandex Object Storage.

Преимущества такого подхода:
- Круглосуточная работа бота без зависимости от локального компьютера.
- Удобная интеграция с S3-хранилищем внутри одной облачной экосистемы.
- Возможность масштабирования — увеличение ресурсов ВМ при росте нагрузки.

Удачи и приятной готовки! 🍳
